In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Vaini,TO,2022-08-07 22:07:25,-21.2000,-175.2000,80.76,89,20,8.05
1,1,Belmonte,BR,2022-08-07 22:07:25,-15.8631,-38.8828,72.79,75,15,8.59
2,2,Albany,US,2022-08-07 22:04:28,42.6001,-73.9662,90.64,69,64,1.01
3,3,Krasnoselkup,RU,2022-08-07 22:07:26,65.7000,82.4667,58.71,78,100,8.23
4,4,Rikitea,PF,2022-08-07 22:02:46,-23.1203,-134.9692,72.52,69,100,32.12


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [9]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
#fig = gmaps.figure()
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],
                                 dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Vaini,TO,2022-08-07 22:07:25,-21.2000,-175.2000,80.76,89,20,8.05
5,5,Puerto Ayora,EC,2022-08-07 22:07:26,-0.7393,-90.3518,75.16,88,62,4.00
9,9,Georgetown,MY,2022-08-07 22:03:26,5.4112,100.3354,80.89,87,20,0.00
11,11,Cantanhede,BR,2022-08-07 22:07:28,-3.6333,-44.3767,80.13,55,1,3.38
12,12,Kavaratti,IN,2022-08-07 22:07:28,10.5669,72.6420,80.49,83,99,23.89
17,17,Hambantota,LK,2022-08-07 22:07:30,6.1241,81.1185,77.34,100,0,10.36
29,29,Komsomolskiy,UZ,2022-08-07 22:07:34,40.4272,71.7189,79.23,25,6,2.98
30,30,Umm Kaddadah,SD,2022-08-07 22:07:35,13.6017,26.6876,84.99,49,94,10.40
34,34,Naze,JP,2022-08-07 22:07:36,28.3667,129.4833,79.65,77,2,2.28
35,35,Hilo,US,2022-08-07 22:07:36,19.7297,-155.0900,89.28,74,75,8.05


In [18]:
preferred_cities_df.notnull().sum()

City_ID       204
City          204
Country       204
Date          204
Lat           204
Lng           204
Max Temp      204
Humidity      204
Cloudiness    204
Wind Speed    204
dtype: int64

In [19]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,-21.2000,-175.2000,
5,Puerto Ayora,EC,75.16,-0.7393,-90.3518,
9,Georgetown,MY,80.89,5.4112,100.3354,
11,Cantanhede,BR,80.13,-3.6333,-44.3767,
12,Kavaratti,IN,80.49,10.5669,72.6420,
17,Hambantota,LK,77.34,6.1241,81.1185,
29,Komsomolskiy,UZ,79.23,40.4272,71.7189,
30,Umm Kaddadah,SD,84.99,13.6017,26.6876,
34,Naze,JP,79.65,28.3667,129.4833,
35,Hilo,US,89.28,19.7297,-155.0900,


In [20]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [22]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    try:
    # Grab the first hotel from the results and store the name.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("it worked")
    except (IndexError):
        print("Hotel not found... skipping.")

it worked
it worked
it worked
it worked
it worked
it worked
it worked
Hotel not found... skipping.
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
Hotel not found... skipping.
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
Hotel not found... skipping.
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
Hotel not found... skipping.
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
Hotel not found... skipping.
it worked
it worked
it worked
it worked
it worked
it worked
Hotel not found... skipping.
it worked
it worked
it worked
it worked
it wor

In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [30]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,-21.2000,-175.2000,Keleti Beach Resort
5,Puerto Ayora,EC,75.16,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Georgetown,MY,80.89,5.4112,100.3354,Cititel Penang
11,Cantanhede,BR,80.13,-3.6333,-44.3767,HOTEL FE EM DEUS
12,Kavaratti,IN,80.49,10.5669,72.6420,Green leaf
17,Hambantota,LK,77.34,6.1241,81.1185,Bungalow 63
29,Komsomolskiy,UZ,79.23,40.4272,71.7189,Ikathouse
30,Umm Kaddadah,SD,84.99,13.6017,26.6876,
34,Naze,JP,79.65,28.3667,129.4833,Shishime Hotel
35,Hilo,US,89.28,19.7297,-155.0900,Hilo Hawaiian Hotel


In [26]:
hotel_df.count()

City          204
Country       204
Max Temp      204
Lat           204
Lng           204
Hotel Name    204
dtype: int64